<a href="https://colab.research.google.com/github/TirendazAcademy/LangChain-Tutorials/blob/main/Quickstart_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Tutorial

This tutorial walks you through how to use LangChain.

# Installation

In [ ]:
!pip install langchain 
!pip install openai 
!pip install google-search-results 

# Importing the Necessary Libraries

In [13]:
import os
from langchain import ConversationChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (PromptTemplate, 
                               ChatPromptTemplate, 
                               MessagesPlaceholder, 
                               SystemMessagePromptTemplate, 
                               HumanMessagePromptTemplate)
from langchain.chains import LLMChain
from langchain.agents import (load_tools, 
                              initialize_agent, 
                              AgentType)
from langchain.schema import (AIMessage, 
                              HumanMessage, 
                              SystemMessage)
from langchain.prompts.chat import (ChatPromptTemplate, 
                                    SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate)
from langchain.memory import ConversationBufferMemory

# Environment Setup

In [ ]:
os.environ["OPENAI_API_KEY"] = "..."
os.environ["SERPAPI_API_KEY"] = "..."

# LLMs: Get predictions from a language model

In [14]:
llm = OpenAI(temperature=0.9)

In [15]:
text = "Suggest a youtube name for AI"
print(llm(text))



AI Wiz


# Prompt Templates: Manage prompts for LLMs

In [40]:
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Recommend a youtube name for {topic}?",
)

In [19]:
print(prompt.format(topic="AI"))

Recommend a youtube name for AI?


In [20]:
chain = LLMChain(llm=llm, prompt=prompt)

In [21]:
chain.run("data science")

'\n\nDataScienceTube'

# Agents: Dynamically Call Chains Based on User Input

In [42]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")

 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 69.8ºf @3:45 PM Low: 55.04ºf @5:56 AM Approx.
Thought: I need to use the calculator to raise 69.8 to the .023 power
Action: Calculator
Action Input: 69.8^.023
Observation: Answer: 1.1025763550530143
Thought: I now know the final answer
Final Answer: 1.1025763550530143

> Finished chain.


'1.1025763550530143'

# Memory: Add State to Chains and Agents

In [25]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)

Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. How can I help you today?


In [26]:
output = conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
print(output)

Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.
 That's great! It's always nice to have a conversation with someone new. What would you like to talk about?


# Get Message Completions from a Chat Model

In [27]:
chat = ChatOpenAI(temperature=0)

In [29]:
chat([HumanMessage(content="Translate this sentence from English to Turkish. I love programming.")])

AIMessage(lc_kwargs={'content': 'Ben programlamayı seviyorum.'}, content='Ben programlamayı seviyorum.', additional_kwargs={}, example=False)

In [30]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Turkish."),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(lc_kwargs={'content': 'Ben programlamayı seviyorum.'}, content='Ben programlamayı seviyorum.', additional_kwargs={}, example=False)

In [31]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(lc_kwargs={'message': AIMessage(lc_kwargs={'content': "J'adore la programmation."}, content="J'adore la programmation.", additional_kwargs={}, example=False)}, text="J'adore la programmation.", generation_info=None, message=AIMessage(lc_kwargs={'content': "J'adore la programmation."}, content="J'adore la programmation.", additional_kwargs={}, example=False))], [ChatGeneration(lc_kwargs={'message': AIMessage(lc_kwargs={'content': "J'adore l'intelligence artificielle."}, content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False)}, text="J'adore l'intelligence artificielle.", generation_info=None, message=AIMessage(lc_kwargs={'content': "J'adore l'intelligence artificielle."}, content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'}, run=RunInfo(run_id=UUID('cd379524-dc

In [32]:
result.llm_output['token_usage']

{'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}

# Chat Prompt Templates

In [33]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", 
                               output_language="Turkish", 
                               text="I love programming.").to_messages())

AIMessage(lc_kwargs={'content': 'Ben programlamayı seviyorum.'}, content='Ben programlamayı seviyorum.', additional_kwargs={}, example=False)

# Chains with Chat Models

In [34]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Turkish", text="I love programming.")

'Ben programlamayı seviyorum.'

# Agents with Chat Models

In [35]:
# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")

Thought: I need to use a search engine to find Olivia Wilde's boyfriend and a calculator to raise his age to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```

Observation: Looks like Olivia Wilde and Jason Sudeikis are starting 2023 on good terms. Amid their highly publicized custody battle – and the actress' ...
Thought:Now I need to use a calculator to raise Jason Sudeikis' age to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "47^0.23"
}
```


Observation: Answer: 2.4242784855673896
Thought:I now know that Jason Sudeikis' current age raised to the 0.23 power is 2.4242784855673896.
Final Answer: Jason Sudeikis' current age raised to the 0.23 power is 2.4242784855673896.

> Finished chain.


"Jason Sudeikis' current age raised to the 0.23 power is 2.4242784855673896."

# Memory: Add State to Chains and Agents

In [38]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")
# -> 'Hello! How can I assist you today?'


conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
# -> "That sounds like fun! I'm happy to chat with you. Is there anything specific you'd like to talk about?"

conversation.predict(input="Tell me about yourself.")

'I am an AI language model designed to assist with various tasks such as answering questions, generating text, and providing recommendations. I am constantly learning and improving my abilities through machine learning algorithms. Is there anything specific you would like to know about me?'

# Resource

- [LangChain Doc](https://python.langchain.com/en/latest/getting_started/getting_started.html#environment-setup)